In [31]:
import warnings
from importlib import reload

with warnings.catch_warnings():
    ## ignore future warnings from too-advanced numpy version
    warnings.simplefilter("ignore")    
    from keras.models import Sequential
    from keras.layers import Dense, Activation
import pandas as pd, numpy as np

from sklearn import preprocessing

In [14]:
import energy
from energy import MissingStrategy
_ = reload(energy)

In [3]:
MAX_ROWS = 100000
weather = energy.Data.Weather(MAX_ROWS)
meter = energy.Data.Meter(MAX_ROWS)
buildings = energy.Data.Buildings()

In [4]:
combined_df = energy.merge_data(
    weather_df=weather.train, 
    meter_df=meter.train,
    buildings_df=buildings.data)

In [54]:
combined_df.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,0,2016-01-01 00:00:00,0.0,0,Education,7432,2008.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
1,1,0,2016-01-01 00:00:00,0.0,0,Education,2720,2004.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
2,2,0,2016-01-01 00:00:00,0.0,0,Education,5376,1991.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
3,3,0,2016-01-01 00:00:00,0.0,0,Education,23685,2002.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
4,4,0,2016-01-01 00:00:00,0.0,0,Education,116607,1975.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0


In [77]:
weather_cols = [
    'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr', 
    'sea_level_pressure', 'wind_direction', 'wind_speed']
X_df = combined_df[[
    'square_feet', 'meter', 'year_built', 'primary_use'] +
    weather_cols]

le = preprocessing.LabelEncoder()
le.fit(list(X_df.loc[:, 'primary_use']))
X_df.loc[:, 'primary_use'] = le.transform(list(X_df['primary_use']))

In [78]:
X = X_df.values
y = combined_df[['meter_reading']].values
assert not np.isnan(y).any()
rmsle = energy.define_rmsle(X_df.shape[0])

In [79]:
missing_strategy = energy.MissingStrategy.USE_COLUMN_MEAN
X, y = energy.handle_missing_predictor_values(X, y, missing_strategy)
assert not np.isnan(X).any()

In [80]:
X_scaled = preprocessing.scale(X)

In [81]:
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=X.shape[1]))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='relu'))

In [82]:
model.compile(optimizer='rmsprop',
              loss='mse', metrics=['mse'])

In [83]:
model.fit(X_scaled, y, epochs=10, batch_size=32)

Epoch 1/10
62225/62225 [==============================] - 3s 54us/step - loss: 436057.5231 - mse: 436057.4688
Epoch 2/10
62225/62225 [==============================] - 3s 51us/step - loss: 414002.0131 - mse: 414002.0625
Epoch 3/10
62225/62225 [==============================] - 3s 52us/step - loss: 411293.3789 - mse: 411293.4062
Epoch 4/10
62225/62225 [==============================] - 3s 50us/step - loss: 408398.7448 - mse: 408398.7500
Epoch 5/10
62225/62225 [==============================] - 3s 50us/step - loss: 405899.1232 - mse: 405899.1250
Epoch 6/10
62225/62225 [==============================] - 3s 50us/step - loss: 402822.8112 - mse: 402822.9375
Epoch 7/10
62225/62225 [==============================] - 3s 50us/step - loss: 400350.7037 - mse: 400350.7188
Epoch 8/10
62225/62225 [==============================] - 3s 51us/step - loss: 397598.0509 - mse: 397598.5000
Epoch 9/10
62225/62225 [==============================] - 3s 52us/step - loss: 394960.5000 - mse: 394960.5625
Epoch 10/1